# HW5 - 爬取網路新聞
#### 【Python人工智慧實戰_2020-05-13】

### Use Crawl to collect url link in "即時看板"
url: https://money.udn.com/money/index

In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import os
from urllib.request import urlretrieve
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
news_list = []
max_retry = 3 # 最多試 3 次
retry = 1

url = 'https://money.udn.com/money/index'

## write your code here
while retry <= max_retry:
    try:
        res = requests.get(url) # 404 Error 不會到 except，要網路斷線才會跑
        res.encoding = 'utf-8'       
        sleep(2) # 【重要】讓電腦停滯 2 秒，避免不斷請求被視為攻擊
        
        soup = BeautifulSoup(res.text, 'html.parser')
        board = soup.find("div", {"class": "tabs_box_wrapper"})
        
        for index, a in enumerate(board.find_all("a")):           
            link_title = a.text
            link_path = 'https://money.udn.com' + a['href']
            
            # 僅爬取文章，不含影音
            if("video" not in link_path):
                news_list.append(link_path)
                print("title: {}, link: {}".format(link_title, link_path))          

        break
    except Exception as e:
        retry += 1 # 若重試成功則還原預設
        print("error: {}".format(e))
        continue # 未達 max_retry，回到最近的迴圈繼續執行

title: 21:21同婚立法一週年 蘇貞昌：繼續讓台灣更好, link: https://money.udn.com/money/story/7307/4570632
title: 21:17俄防疫負責人：全國疫情趨穩 但要規劃夏季出遊仍太早, link: https://money.udn.com/money/story/5599/4570625
title: 21:12以色列總理提出新內閣名單 敦促兼併西岸屯墾區, link: https://money.udn.com/money/story/5599/4570621
title: 21:12美國管制華為 中國：採取一切必要措施維權, link: https://money.udn.com/money/story/5599/4570622
title: 20:45陳菊請辭總統府秘書長 獲提名監察院長呼聲高, link: https://money.udn.com/money/story/7307/4570577
title: 20:36中國駐以色列大使猝逝 媒體指死因疑為心臟問題, link: https://money.udn.com/money/story/5599/4570557
title: 20:22世衛專家：兒童染疫風險非常低　若落實社交距離可復課, link: https://money.udn.com/money/story/5599/4570509
title: 20:21陳菊不續任府秘書長 總統：不捨但尊重意願, link: https://money.udn.com/money/story/7307/4570500
title: 20:17西班牙新冠肺炎單日增87死 兩個月來首度低於百死, link: https://money.udn.com/money/story/5599/4570473
title: 12:08【名家觀點】雲上創新是企業轉型關鍵, link: https://money.udn.com/money/story/12952/4569794
title: 07:30活不活看周轉靈不靈 企業小蝦米的決戰時刻！, link: https://money.udn.com/money/story/12989/4567125
title: 07:30【經濟彭博周報】投入防疫生

### Save each new in a folder(including content and image)

In [3]:
# 建立資料夾
def createFolder(directory):
    try:       
        # not True 為 False，not False 為 True
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [4]:
from urllib.parse import urlparse

# 檢查合法路徑
def is_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

In [5]:
data = []
for news_url in news_list:
    ## write your code here
    while retry <= max_retry:
        try:
            arr = []
            res = requests.get(news_url)
            res.encoding = 'utf-8'          
            sleep(2)
            print("news url: {}".format(news_url))
            soup = BeautifulSoup(res.text, 'html.parser')
            content = soup.find("div", {"id": "story_body_content"})
            title = content.find("h2", {"id": "story_art_title"}).text
            article_body = content.find("div", {"id": "article_body"})
            news_dir = 'news/' + title            
            createFolder(news_dir)
            
            print("news title: {}".format(title))
            arr.append(title)
           
            # get article
            str_article = str()
            file_path = news_dir + '/article.txt'
            with open(file_path, 'w', encoding='utf-8') as fp:
                print("news content:")
                if(len(article_body.find_all("p")) > 0):
                    for idx, p in enumerate(article_body.find_all("p")):
                        text = p.text.strip()
                        str_article += text
                        print(text)
                        fp.write(text + '\r\n')
                    arr.append(str_article)
                  
            # get imgs
            str_img = str()
            if(len(article_body.find_all("figure")) > 0):
                print("save image:") 
                for idx, fig in enumerate(article_body.find_all("figure")):
                    fig_url = fig.find("img")['src']
                  
                    if(is_url(fig_url)):
                        print(fig_url)                     
                        ext = os.path.splitext(fig_url.split("&", 1)[0])[1]
                        img_path = news_dir + '/img_' + str(idx) + ext
                        str_img += str(fig_url + "\r\n")                       
                        urlretrieve(fig_url, img_path) # urlretrieve 下載檔案
                arr.append(str_img)
            
            print("==========" + "\n\n")           
            data.append(arr)
            
            break
        except Exception as e:
            retry += 1
            print("error: {}".format(e))
            continue

news url: https://money.udn.com/money/story/7307/4570632
news title: 同婚立法一週年 蘇貞昌：繼續讓台灣更好
news content:
台灣同性婚姻合法化今天立法滿一週年，行政院長蘇貞昌指出「一年前，因為我們一起勇敢。一年後的今天，我們才能成為一個讓世界看到、讓人民驕傲的國家。」他期待，民眾一起帶著那時的經驗、智慧與熱情，繼續讓台灣更好。
立法院會去年的5月17日三讀通過司法院釋字第748號解釋施行法，保障相同性別兩人可向戶政機關辦理結婚登記，這也讓台灣成為亞洲第一個同性婚姻法制化國家。
蘇貞昌晚間透過臉書發文，分享一部影片。這是去年專法審查前一天，蘇貞昌對執政黨立委說的一席話。
影片中，蘇貞昌對立委們說，立法院通過的法案幾百、幾千件，但將來可以拿出來談的、拿出來講的，其實不多。將來一定會被問到的，一定有明天同婚這一案。
蘇貞昌表示，到那時候，整個社會進步到那裡了，立委可以拍胸脯、很大聲地說：「你看，當年壓力是多麼大，但我還是頂下來了，我投下關鍵的一票！」
蘇貞昌今天說，「一年前，因為我們一起勇敢。一年後的今天，我們才能成為一個讓世界看到、讓人民驕傲的國家。」
他說，一年前的今天，同婚專法在立院三讀，台灣人展現智慧與包容，一起把這個國家往前推了一步。「但我們知道，這只是個開始」。這一年來，政府努力以各種方式做溝通、促進理解，不但讓同性伴侶得到公平的法律保障，也確實讓更多人能接納不同、善待彼此。
蘇貞昌說，他想藉由一週年的原音重現，期待所有民眾，一起帶著那時的經驗、智慧與熱情，繼續讓台灣更好、讓政府對所有人民更周到、讓國家繼續進步下去。


news url: https://money.udn.com/money/story/5599/4570625
news title: 俄防疫負責人：全國疫情趨穩 但要規劃夏季出遊仍太早
news content:
負責防疫工作的俄羅斯聯邦消費者權益保護監督局局長波波娃17日說，該國新冠肺炎疫情正趨於穩定，單日確診人數在本周第三度低於1萬人。俄國17日新增9709起病例，目前累計28萬1752例，在全世界僅次於美國。

波波娃在接受俄國電視台「第一頻道」（Rossiya-1）專訪時說：「我們已經朝我們一直期待的穩定水位邁進。在過去10到11天裡，我們沒有看到額

In [6]:
df = pd.DataFrame(data, columns = ["Title", "Content", "ImgUrl"]) # 指定標籤欄名稱
df.to_csv('news.csv', index=False, encoding="utf_8_sig")
df.head(30)

,Title,Content,ImgUrl
0,同婚立法一週年 蘇貞昌：繼續讓台灣更好,台灣同性婚姻合法化今天立法滿一週年，行政院長蘇貞昌指出「一年前，因為我們一起勇敢。一年後的今...,None
1,俄防疫負責人：全國疫情趨穩 但要規劃夏季出遊仍太早,負責防疫工作的俄羅斯聯邦消費者權益保護監督局局長波波娃17日說，該國新冠肺炎疫情正趨於穩定，...,https://pgw.udn.com.tw/gw/photo.php?u=https://...
2,以色列總理提出新內閣名單 敦促兼併西岸屯墾區,以色列總理尼坦雅胡今天向國會提交新聯合政府組閣名單，為持續超過一年的政治僵局畫上句點。他強調...,None
3,美國管制華為 中國：採取一切必要措施維權,美國政府15日公布出口管制新規，限制華為使用美國技術發展半導體的能力，中國商務部今天說，此舉...,None
4,陳菊請辭總統府秘書長 獲提名監察院長呼聲高,總統府秘書長陳菊在黨內輩分高，始終扮演「桶箍」角色，但也常是「石磨心」，忍辱負重，如今將離開...,None
5,中國駐以色列大使猝逝 媒體指死因疑為心臟問題,中國駐以色列大使杜偉今天被人發現死於特拉維夫市（Tel Aviv）郊外的官邸。以色列媒體引述...,None
6,世衛專家：兒童染疫風險非常低 若落實社交距離可復課,世界衛生組織（WHO）首席科學家史瓦米娜森博士17日表示，若充分落實社交距離措施，學校可重新...,https://pgw.udn.com.tw/gw/photo.php?u=https://...
7,陳菊不續任府秘書長 總統：不捨但尊重意願,520前夕，總統府秘書長陳菊今天表示，520以後不再續任總統府秘書長。總統蔡英文透過臉書回應...,None
8,西班牙新冠肺炎單日增87死 兩個月來首度低於百死,路透報導，西班牙衛生部17日表示，境內新增新冠肺炎87例死亡，是2個月來單日病故人數首度低於...,None
9,【名家觀點】雲上創新是企業轉型關鍵,近年來，人工智慧、5G、互聯網等新興技術方興未艾，再加上今年因新冠肺炎疫情影響，讓遠程醫療、...,None
